In [ ]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/EE-451-IAPR/project/src

/content/drive/MyDrive/EE-451-IAPR/project/src


# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [10]:
#Import libraries
import torch
import importlib

import warnings
warnings.filterwarnings("ignore")

In [11]:
#Import files
from data_classes.ref_data import refCoin
from data_classes.train_data import trainCoin
from data_classes.test_data import testCoin
import constants
from visualization import *
from pickle_func import *
importlib.reload(constants)
from processing.process_func import *
from models.utils import *
from processing.data_augmentation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load different Datasets

In [ ]:
ref_data = refCoin()

In [ ]:
train_data = trainCoin(save=False, load_from_pickle=False)

Loading data from pickle files


In [ ]:
test_data = testCoin()

Loading data from pickle files


# 2 Create Data Ready for Neural Network

In [ ]:
#Process all the images :
# 1. clean the images and find the contours
# 2. create the masked images
# 3. create the coin images
# 4. if save = True, save the images and the class in a result folder
train_data.proceed_data()

Finding contours
Creating masked images
Creating coin images


In [ ]:
save_pickle(train_data.coins, 'coins.pkl')
save_pickle(train_data.contours, 'contours.pkl')

In [12]:
conversion_table = get_classes_conv_table()

In [ ]:
# Associate the labels to the coins
coin_labels = get_coin_labels()

In [ ]:
# Extract the images and the labels + create a dataframe that summarize the data
images, labels, df_images_labels = create_data_structure(train_data.coins, train_data.contours, coin_labels, conversion_table)

In [ ]:
save_coins_classified(df_images_labels, images)

In [ ]:
train_images, train_labels, val_images, val_labels = create_splits(images, labels)

# 3 Data Augmentation

In [ ]:
train_images_aug, train_labels_aug = augment_set(train_images, train_labels)

In [ ]:
train_images_aug, train_labels_aug = augment_blur(train_images_aug, train_labels_aug)

In [ ]:
train_images_aug_resized = resize_images(train_images_aug, 200)

In [ ]:
save_pickle(train_images_aug_resized, 'train_images_aug_resized.pkl')

In [13]:
train_images_aug_resized = load_pickle('train_images_aug_resized.pkl')
train_images_labels_aug = load_pickle('train_labels_aug.pkl')

val_images_resized = load_pickle('val_images_resized.pkl')
val_images_labels = load_pickle('val_labels.pkl')

# 4 Train Neural Network


In [14]:
train_dataloader, val_dataloader = create_dataloader(
    train_images_aug_resized,
    train_images_labels_aug,
    val_images_resized,
    val_images_labels)

## 4.1 Custom Basic CNN

In [15]:
from models.cnn import Basic_CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

image_dim = train_images_aug_resized.shape[1]
num_classes = len(conversion_table)
# Define the model
cnn = Basic_CNN(image_size=image_dim, num_classes=num_classes)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

Epoch 1/10, Loss: 2.2964949739092395, Validation Accuracy: 0.6883
Epoch 2/10, Loss: 1.718103261017106, Validation Accuracy: 0.7403
Epoch 3/10, Loss: 0.7807000739371538, Validation Accuracy: 0.9221
Epoch 4/10, Loss: 2.0474204140315675, Validation Accuracy: 0.8442
Epoch 5/10, Loss: 2.791350682638119, Validation Accuracy: 0.7922
Epoch 6/10, Loss: 6.755259082910292, Validation Accuracy: 0.7532
Epoch 7/10, Loss: 4.072394247674139, Validation Accuracy: 0.8182
Epoch 8/10, Loss: 4.198189846734584, Validation Accuracy: 0.8052
Epoch 9/10, Loss: 4.379997418846135, Validation Accuracy: 0.7922
Epoch 10/10, Loss: 4.464120910213626, Validation Accuracy: 0.7922


# 4.2 RESNET-50

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from torchvision import transforms
from datasets import load_metric

# Step 2: Modify the final layer
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Replace the classifier
num_classes = 15
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Step 3: Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    model.train()
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item()}")

# Evaluation
model.eval()
metric = load_metric("accuracy")
for images, labels in val_dataloader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images).logits
    predictions = torch.argmax(outputs, dim=1)
    metric.add_batch(predictions=predictions, references=labels)

accuracy = metric.compute()
print(f"Test Accuracy: {accuracy['accuracy']:.4f}")
